In [2]:
from copy import deepcopy
from pathlib import Path
import cv2
import matplotlib.pylab as plt
import numpy as np
from scipy.io import loadmat
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
import cv2
import pandas as pd
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 

the dataframes come from IDMatcher-PerFrame.ipynb

All methodes use 1 epipolar geometry made using a random frame.

for df_preds per frame we created a epipolar line per left id and chose the known right id with the lowest euclidean distance to this epipolor line as the "matching" id. This results in certain right_ids occuring more than once in frames. 

For one measure we Then chose the most commonly predicted right id among all frames combined for each left id as its most likely correct match.

For the second measure we calculated the correctness per frame and averaged this out.

For pos_match we took all assumed matches of each left id amongs all frames, created a top three most common ones, calculated the average difference using the difference when predicting from left to right and right to left combined from the predicted points to the "chosen" true points. Used this to determen from the top 3 which ones was the "best" match per id based on the lowest average distance which resulted in result_df. This resulted in some right_ids occuring more than once tho. 

In [153]:
df_preds = pd.read_csv('df_preds_v2.csv')
df_predsV4 = pd.read_csv('df_preds_V4.csv')
pos_match = pd.read_csv('pos_match_v2.csv')
result_df = pd.read_csv('result_df_v2.csv')
csv_path_1 = '129_1_clean.txt'
df1 = pd.read_csv(csv_path_1)
csv_path_2 = '129_2_clean.txt'
df2 = pd.read_csv(csv_path_2)

In [66]:
gt_matches = {3:0, 5:1, 4:2, 2:3, 8:4, 0:5, 1:6, 6:7, 7:8, 9:9, 10:10, 11:11, 12:12, 13:13, 14:14}
display(df_preds)
# display(pos_match)
display(result_df)

,left_id,right_id,difference,frame
0,0.0,5.0,27.999137,1.0
1,1.0,5.0,4.070687,1.0
2,2.0,3.0,2.869824,1.0
3,3.0,0.0,4.092777,1.0
4,4.0,2.0,6.089725,1.0
...,...,...,...,...
30196,4.0,10.0,3.773182,3117.0
30197,5.0,1.0,12.651285,3117.0
30198,6.0,7.0,10.728657,3117.0
30199,7.0,10.0,3.773182,3117.0


,left_right,left_boy,right_boy,avg_diff,left_right_diff,right_left_diff
0,"(1.0, 8.0)",0.0,7.0,39.212002,31.038243,8.173759
1,"(10.0, 10.0)",9.0,9.0,16.562931,6.784330,9.778601
2,"(11.0, 1.0)",10.0,0.0,6.069236,2.411325,3.657911
3,"(12.0, 2.0)",11.0,1.0,10.654859,1.745077,8.909782
4,"(13.0, 13.0)",12.0,12.0,12.415106,4.173788,8.241318
5,"(14.0, 14.0)",13.0,13.0,52.197162,2.936132,49.261030
6,"(15.0, 15.0)",14.0,14.0,66.361335,12.157227,54.204109
7,"(2.0, 7.0)",1.0,6.0,55.052347,28.305855,26.746493
8,"(3.0, 4.0)",2.0,3.0,38.985178,3.520585,35.464594
9,"(4.0, 12.0)",3.0,11.0,7.091647,2.568191,4.523456


In [54]:
# Group by 'left_id' and count occurrences of each 'right_id'
most_common_right_id = df_preds.groupby('left_id')['right_id'].agg(lambda x: x.value_counts().index[0])

# Convert Series to DataFrame
most_common_df = most_common_right_id.reset_index(name='right_id')

most_common_df

,left_id,right_id
0,0.0,5.0
1,1.0,6.0
2,2.0,3.0
3,3.0,0.0
4,4.0,2.0
5,5.0,1.0
6,6.0,7.0
7,7.0,9.0
8,8.0,4.0
9,9.0,9.0


In [157]:
# subset of 298 frames version with epipolar per frame

# Group by 'left_id' and count occurrences of each 'right_id'
most_common_right_id_v2 = df_predsV4.groupby('left_id')['right_id'].agg(lambda x: x.value_counts().index[0])

# Convert Series to DataFrame
most_common_df_v2 = most_common_right_id_v2.reset_index(name='right_id')

display(most_common_df_v2)


# subset of 298 frames version with epipolar once

df_preds_normal_for_comp = df_preds.loc[df_preds['frame'] < 298]

# Group by 'left_id' and count occurrences of each 'right_id'
most_common_right_id_v3 = df_preds_normal_for_comp.groupby('left_id')['right_id'].agg(lambda x: x.value_counts().index[0])

# Convert Series to DataFrame
most_common_df_v3 = most_common_right_id_v3.reset_index(name='right_id')

display(most_common_df_v3)


def performance_matches(df,matches):
    # based on most common matches across all frames
    matches_count_m1 = 0
    for x in range(len(df)):
        l = df['left_id'][x]
        r = df['right_id'][x]
        if matches[l] == r:
            matches_count_m1 += 1
    print(matches_count_m1/len(df))
    
performance_matches(most_common_df_v3,gt_matches)
performance_matches(most_common_df_v2,gt_matches)

,left_id,right_id
0,0.0,5.0
1,1.0,6.0
2,2.0,3.0
3,3.0,0.0
4,4.0,2.0
5,5.0,2.0
6,6.0,7.0
7,7.0,7.0
8,8.0,4.0


,left_id,right_id
0,0.0,5.0
1,1.0,5.0
2,2.0,3.0
3,3.0,2.0
4,4.0,2.0
5,5.0,2.0
6,6.0,7.0
7,7.0,7.0
8,8.0,4.0


0.5555555555555556
0.7777777777777778


performance is better when doing epipolar per frame, difference of 22.22% on subset of 298 frames

In [53]:
print(gt_matches)

{3: 0, 5: 1, 4: 2, 2: 3, 8: 4, 0: 5, 1: 6, 6: 7, 7: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}


The method below calculates corectness by picking the most commonly predicted right id per left id and setting that as a match.

In [159]:
# based on most common matches across all frames
matches_count_m1 = 0
for x in range(len(most_common_df)):
    l = most_common_df['left_id'][x]
    r =  most_common_df['right_id'][x]
    if gt_matches[l] == r:
        matches_count_m1 += 1
        
print(matches_count_m1/len(most_common_df))

0.8


In [64]:
result_df

,left_right,left_boy,right_boy,avg_diff,left_right_diff,right_left_diff
0,"(1.0, 8.0)",0.0,7.0,39.212002,31.038243,8.173759
1,"(10.0, 10.0)",9.0,9.0,16.562931,6.784330,9.778601
2,"(11.0, 1.0)",10.0,0.0,6.069236,2.411325,3.657911
3,"(12.0, 2.0)",11.0,1.0,10.654859,1.745077,8.909782
4,"(13.0, 13.0)",12.0,12.0,12.415106,4.173788,8.241318
5,"(14.0, 14.0)",13.0,13.0,52.197162,2.936132,49.261030
6,"(15.0, 15.0)",14.0,14.0,66.361335,12.157227,54.204109
7,"(2.0, 7.0)",1.0,6.0,55.052347,28.305855,26.746493
8,"(3.0, 4.0)",2.0,3.0,38.985178,3.520585,35.464594
9,"(4.0, 12.0)",3.0,11.0,7.091647,2.568191,4.523456


the method below calculates correctnes using the matches from result_df

In [158]:
l_boys = list(result_df['left_boy'])
r_boys = list(result_df['right_boy'])

df_avg_dif_based = pd.DataFrame({'left_id': l_boys, 'right_id': r_boys})
display(df_avg_dif_based)
# based on best matches based on lowest average distance from predicted match to true match
matches_count_m2 = 0
for x in range(len(df_avg_dif_based )):
    l = df_avg_dif_based ['left_id'][x]
    r =  df_avg_dif_based ['right_id'][x]
    if gt_matches[l] == r:
        print(l,r)
        matches_count_m2 += 1
        
print(matches_count_m2/len(df_avg_dif_based))
print(gt_matches)

,left_id,right_id
0,0.0,7.0
1,9.0,9.0
2,10.0,0.0
3,11.0,1.0
4,12.0,12.0
5,13.0,13.0
6,14.0,14.0
7,1.0,6.0
8,2.0,3.0
9,3.0,11.0


9.0 9.0
12.0 12.0
13.0 13.0
14.0 14.0
1.0 6.0
2.0 3.0
4.0 2.0
0.4666666666666667
{3: 0, 5: 1, 4: 2, 2: 3, 8: 4, 0: 5, 1: 6, 6: 7, 7: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14}


The method below calculates correctness of matches per frame. then averages this out amongst all frames.

In [144]:
# based on amount of matches per frame
match_count_dict = dict()
for frame in set(df_preds['frame']):
    match_count = 0
    temp = df_preds.loc[df_preds['frame'] == frame]
#     print(list(temp['left_id']),list(temp['right_id']))
    left_l = list(temp['left_id'])
    right_l = list(temp['right_id'])
    for x in range(len(left_l)):
#         print(left_l[x],right_l[x])
        if gt_matches[left_l[x]] == right_l[x]:
#             print("MATCH",left_l[x],right_l[x],frame)
            match_count += 1
    match_count_dict[frame] = match_count/len(left_l)
# match_count_dict

average_value_matches = sum(match_count_dict.values()) / len(match_count_dict)
print(average_value_matches)
# print(match_count_dict)

0.667581481007858


In [152]:
# V2 with epipolar per frame on subset of 298 frames
df_predsV4 = pd.read_csv('df_preds_V4.csv')
df_preds_normal_for_comp = df_preds.loc[df_preds['frame'] < 298]
display(df_preds_normal_for_comp)
display(df_predsV4)

def measure_correctness(df,matches):
    match_count_dict = dict()
    for frame in set(df['frame']):
        match_count = 0
        temp = df.loc[df['frame'] == frame]
    #     print(list(temp['left_id']),list(temp['right_id']))
        left_l = list(temp['left_id'])
        right_l = list(temp['right_id'])
        for x in range(len(left_l)):
    #         print(left_l[x],right_l[x])
            if matches[left_l[x]] == right_l[x]:
    #             print("MATCH",left_l[x],right_l[x],frame)
                match_count += 1
        match_count_dict[frame] = match_count/len(left_l)
    # match_count_dict
    average_value_matches = sum(match_count_dict.values()) / len(match_count_dict)
    print(average_value_matches)
    
measure_correctness(df_preds_normal_for_comp,gt_matches)
measure_correctness(df_predsV4,gt_matches)

,left_id,right_id,difference,frame
0,0.0,5.0,27.999137,1.0
1,1.0,5.0,4.070687,1.0
2,2.0,3.0,2.869824,1.0
3,3.0,0.0,4.092777,1.0
4,4.0,2.0,6.089725,1.0
...,...,...,...,...
2668,4.0,1.0,0.842883,297.0
2669,5.0,1.0,0.842883,297.0
2670,6.0,5.0,6.834199,297.0
2671,7.0,6.0,5.872412,297.0


,left_id,right_id,difference,frame
0,0.0,5.0,5.930170,1.0
1,1.0,5.0,5.930170,1.0
2,2.0,3.0,7.923938,1.0
3,3.0,0.0,5.477780,1.0
4,4.0,2.0,6.025641,1.0
...,...,...,...,...
2668,4.0,1.0,1.700299,297.0
2669,5.0,1.0,1.700299,297.0
2670,6.0,7.0,6.409233,297.0
2671,7.0,7.0,6.409233,297.0


0.5798728020950231
0.6213991769547301


when using epipolar on each frame it performs better then when using epipolar once on a random frame, on the subset of 298 frames there is a performance difference of 4%. 

In [74]:
display(df_preds)
display(df1)
display(df2)

,left_id,right_id,difference,frame
0,0.0,5.0,27.999137,1.0
1,1.0,5.0,4.070687,1.0
2,2.0,3.0,2.869824,1.0
3,3.0,0.0,4.092777,1.0
4,4.0,2.0,6.089725,1.0
...,...,...,...,...
30196,4.0,10.0,3.773182,3117.0
30197,5.0,1.0,12.651285,3117.0
30198,6.0,7.0,10.728657,3117.0
30199,7.0,10.0,3.773182,3117.0


,frame,id,x,y,x_offset,y_offset
0,1,1,1212.98,416.74,26.04,14.37
1,1,2,1151.14,415.00,27.58,10.34
2,1,3,1487.32,431.94,17.38,19.86
3,1,4,1080.11,337.38,18.68,6.46
4,1,5,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,5,1011.60,235.99,10.38,22.01
30197,3117,6,1304.99,217.40,15.01,24.80
30198,3117,7,969.60,489.40,23.90,16.40
30199,3117,8,1037.36,245.93,14.89,29.79


,frame,id,x,y,x_offset,y_offset
0,1,1,1010.39,357.55,21.50,7.74
1,1,2,1151.46,337.71,20.11,17.63
2,1,3,1173.87,341.47,11.20,12.02
3,1,4,1446.30,460.60,19.11,20.90
4,1,5,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,5,949.07,696.08,23.25,33.30
30645,3118,6,1067.44,288.15,38.96,21.85
30646,3118,7,1188.06,193.33,33.94,17.14
30647,3118,8,883.91,514.11,25.29,14.69


In [5]:
# Load necessary matrices and coefficients
dd = loadmat("stereoParams_Dep4.mat")
distCoeffs1 = deepcopy(dd["distortionCoefficients1"])
distCoeffs2 = deepcopy(dd["distortionCoefficients2"])
cameraMatrix1 = deepcopy(dd["intrinsicMatrix1"])
cameraMatrix2 = deepcopy(dd["intrinsicMatrix2"])
R = deepcopy(dd["rotationOfCamera2"])
T = deepcopy(dd["translationOfCamera2"])
cameraMatrix1[0:2, 2] += 1
cameraMatrix2[0:2, 2] += 1
# Projection matrices
P1 = np.dot(cameraMatrix1, np.hstack((np.eye(3), np.zeros((3, 1)))))
P2 = np.dot(cameraMatrix2, np.hstack((R, T.reshape(3, 1))))
# df_preds_test = df_preds.loc[df_preds['frame'] == 1]
# df_preds_test

,left_id,right_id,difference,frame
0,1.0,6.0,27.999137,1.0
1,2.0,6.0,4.070687,1.0
2,3.0,4.0,2.869824,1.0
3,4.0,1.0,4.092777,1.0
4,5.0,3.0,6.089725,1.0
5,6.0,3.0,6.089725,1.0
6,7.0,9.0,3.772200,1.0
7,8.0,8.0,10.418142,1.0
8,9.0,5.0,37.689182,1.0


uses df_preds dataframe and matches which score on average 0.67 accuracy

In [6]:
df_3d = pd.DataFrame()
for index, row in df_preds.iterrows():
#     print(row['left_id'])
    df1_temp = df1.loc[(df1['id'] == row['left_id']) & (df1['frame'] == row['frame'])]
    df2_temp = df2.loc[(df2['id'] == row['right_id']) & (df2['frame'] == row['frame'])]
    left_x = float(df1_temp['x'])
    left_y = float(df1_temp['y'])
    left_cor = np.array([left_x,left_y])
    right_x = float(df2_temp['x'])
    right_y = float(df2_temp['y'])
    right_cor = np.array([right_x,right_y])
    left_id = int(df1_temp['id'])
    right_id = int(df2_temp['id'])
#     print(int(row['left_id']),left_id,left_cor,int(row['right_id']),right_id,right_cor)
    # # Undistort points
    undistortedPoints1 = cv2.undistortPoints(
        left_cor, cameraMatrix1, distCoeffs1, P=cameraMatrix1
    )  # result Nx1x2. No need for np.expand_dims(imagePoints1, axis=1)
    undistortedPoints2 = cv2.undistortPoints(
        right_cor, cameraMatrix2, distCoeffs2, P=cameraMatrix2
    )
    
    # Perform triangulation
    point_3d_homogeneous = cv2.triangulatePoints(P1, P2, undistortedPoints1, undistortedPoints2)
    # Convert homogeneous coordinates to Cartesian
    point_3d_cartesian = point_3d_homogeneous[:3] / point_3d_homogeneous[3]
    # Extract x, y, z coordinates
    x, y, z = point_3d_cartesian.flatten()
    
#     new_row_data = {'left_id': left_id, 'right_id': row['id'],'difference': difference,'frame':row['frame']}
#     new_row = pd.DataFrame(new_row_data,index=[0])
    
    df_temp = pd.DataFrame({'frame':int(row['frame']),'left_id': left_id, 'right_id': right_id,'x': x,'y': y, 'z': z},index=[0])
    df_3d = pd.concat([df_3d, df_temp], ignore_index=True)
    
df_3d

/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_21441/3735302214.py:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  left_x = float(df1_temp['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_21441/3735302214.py:7: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  left_y = float(df1_temp['y'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_21441/3735302214.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  right_x = float(df2_temp['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_21441/3735302214.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  

,frame,left_id,right_id,x,y,z
0,1,1,6,475.641316,-251.609266,2175.999486
1,1,2,6,478.872969,-333.698692,2894.645160
2,1,3,4,1187.891829,-270.103752,2587.299296
3,1,4,1,273.150068,-478.047934,2606.051411
4,1,5,3,634.803972,-551.119175,2809.724279
...,...,...,...,...,...,...
30196,3117,5,11,153.236529,-871.108718,3291.131236
30197,3117,6,2,770.093280,-740.882905,2565.283206
30198,3117,7,8,26.176099,-131.034777,2496.313876
30199,3117,8,11,195.927570,-744.434856,2857.989353


In [8]:
df_3d['x'].min()

-23186.503588355223

In [9]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display
df = df_3d
# Group the DataFrame by 'frame'
grouped = df.groupby('frame')

# Create a function to update the plot based on the selected frame
def update_plot(frame):
    data = grouped.get_group(frame)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data['x'], data['y'], data['z'], marker='o')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Frame {frame}')
    plt.show()

# Create a slider widget
frame_slider = widgets.IntSlider(min=df['frame'].min(), max=df['frame'].max(), value=df['frame'].min(), description='Frame')

# Display the slider and plot
widgets.interact(update_plot, frame=frame_slider)


interactive(children=(IntSlider(value=1, description='Frame', max=3117, min=1), Output()), _dom_classes=('widg…

<function __main__.update_plot(frame)>

uses most_common_df which uses most common match among all frames for matches and results in 0.8 accuracy

In [83]:
display(most_common_df)
most_common_dict = dict(zip(most_common_df['left_id'], most_common_df['right_id']))
print(most_common_dict)

,left_id,right_id
0,0.0,5.0
1,1.0,6.0
2,2.0,3.0
3,3.0,0.0
4,4.0,2.0
5,5.0,1.0
6,6.0,7.0
7,7.0,9.0
8,8.0,4.0
9,9.0,9.0


{0.0: 5.0, 1.0: 6.0, 2.0: 3.0, 3.0: 0.0, 4.0: 2.0, 5.0: 1.0, 6.0: 7.0, 7.0: 9.0, 8.0: 4.0, 9.0: 9.0, 10.0: 10.0, 11.0: 11.0, 12.0: 1.0, 13.0: 13.0, 14.0: 13.0}


In [111]:
csv_path_1 = '129_1_clean.txt'
df1 = pd.read_csv(csv_path_1)
csv_path_2 = '129_2_clean.txt'
df2 = pd.read_csv(csv_path_2)
display(df1)
display(df2)
df1['id'] = df1['id'] - 1
df2['id'] = df2['id'] - 1
display(df1)
display(df2)

,frame,id,x,y,x_offset,y_offset
0,1,1,1212.98,416.74,26.04,14.37
1,1,2,1151.14,415.00,27.58,10.34
2,1,3,1487.32,431.94,17.38,19.86
3,1,4,1080.11,337.38,18.68,6.46
4,1,5,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,5,1011.60,235.99,10.38,22.01
30197,3117,6,1304.99,217.40,15.01,24.80
30198,3117,7,969.60,489.40,23.90,16.40
30199,3117,8,1037.36,245.93,14.89,29.79


,frame,id,x,y,x_offset,y_offset
0,1,1,1010.39,357.55,21.50,7.74
1,1,2,1151.46,337.71,20.11,17.63
2,1,3,1173.87,341.47,11.20,12.02
3,1,4,1446.30,460.60,19.11,20.90
4,1,5,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,5,949.07,696.08,23.25,33.30
30645,3118,6,1067.44,288.15,38.96,21.85
30646,3118,7,1188.06,193.33,33.94,17.14
30647,3118,8,883.91,514.11,25.29,14.69


,frame,id,x,y,x_offset,y_offset
0,1,0,1212.98,416.74,26.04,14.37
1,1,1,1151.14,415.00,27.58,10.34
2,1,2,1487.32,431.94,17.38,19.86
3,1,3,1080.11,337.38,18.68,6.46
4,1,4,1220.90,325.30,21.28,10.54
...,...,...,...,...,...,...
30196,3117,4,1011.60,235.99,10.38,22.01
30197,3117,5,1304.99,217.40,15.01,24.80
30198,3117,6,969.60,489.40,23.90,16.40
30199,3117,7,1037.36,245.93,14.89,29.79


,frame,id,x,y,x_offset,y_offset
0,1,0,1010.39,357.55,21.50,7.74
1,1,1,1151.46,337.71,20.11,17.63
2,1,2,1173.87,341.47,11.20,12.02
3,1,3,1446.30,460.60,19.11,20.90
4,1,4,1066.87,626.29,48.07,6.87
...,...,...,...,...,...,...
30644,3118,4,949.07,696.08,23.25,33.30
30645,3118,5,1067.44,288.15,38.96,21.85
30646,3118,6,1188.06,193.33,33.94,17.14
30647,3118,7,883.91,514.11,25.29,14.69


In [118]:
display(df1.loc[df1['frame'] == 2])
display(df2.loc[df2['frame'] == 2])

,frame,id,x,y,x_offset,y_offset
9,2,0,1212.98,416.70,25.70,14.69
10,2,1,1150.88,414.92,27.58,10.35
11,2,2,1487.39,431.99,17.37,19.86
12,2,3,1080.11,337.29,18.68,6.51
13,2,4,1220.85,325.02,21.15,10.62
14,2,5,1211.29,319.44,27.32,9.04
15,2,6,1069.43,499.20,20.92,10.81
16,2,7,1091.36,476.28,17.51,7.76
17,2,8,1190.17,591.21,32.52,9.49


,frame,id,x,y,x_offset,y_offset
9,2,0,1010.38,357.49,21.50,7.74
10,2,1,1151.46,337.71,20.17,17.34
11,2,2,1173.75,341.27,11.20,12.03
12,2,3,1446.41,460.70,19.11,20.90
13,2,4,1066.22,626.24,48.07,6.87
14,2,5,1103.69,442.34,26.32,12.24
15,2,6,1037.62,437.88,37.15,14.25
16,2,7,988.06,528.35,18.49,9.03
17,2,8,978.94,500.22,28.07,8.96


In [114]:
print(most_common_dict)

{0.0: 5.0, 1.0: 6.0, 2.0: 3.0, 3.0: 0.0, 4.0: 2.0, 5.0: 1.0, 6.0: 7.0, 7.0: 9.0, 8.0: 4.0, 9.0: 9.0, 10.0: 10.0, 11.0: 11.0, 12.0: 1.0, 13.0: 13.0, 14.0: 13.0}


if a match is an id that is not in the frame (so a incorrect match) then drop both fish.

potential solution: take the frame, do epipolar geometry, make a new best guess for the match from the list of possible matches based on the frame (either exlude all fish that have been matched already for that frame or dont) make this the new match and continue. 

In [122]:
df_3d_v2 = pd.DataFrame()
for frame in set(df1['frame']):
#     print(frame,"frame")
    df1_temp = df1.loc[df1['frame'] == frame]
    df2_temp = df2.loc[df2['frame'] == frame]
    for cur_id in list(df1_temp['id']):
        df1_tt = df1_temp.loc[df1_temp['id'] == cur_id]
#         df2_tt = df2_temp.loc[df2_temp['id'] == cur_id]
        x_1 = float(df1_tt['x'])
        y_1 = float(df1_tt['y'])
        match_id = most_common_dict[cur_id]
#         print(cur_id,"cur",(match_id),"matchID")
        if match_id in list(df2_temp['id']):
#             print(cur_id,match_id, "pair")
            df2_tt = df2_temp.loc[df2_temp['id'] == match_id]
            x_2 = float(df2_tt['x'])
            y_2 = float(df2_tt['y'])
#             print(x_1,y_1,cur_id,l_id_corrected,match_id,x_2,y_2) 
            left_cor = np.array([x_1,y_1])
            right_cor = np.array([x_2,y_2])
            
            undistortedPoints1 = cv2.undistortPoints(left_cor, cameraMatrix1, distCoeffs1, P=cameraMatrix1)  # result Nx1x2. No need for np.expand_dims(imagePoints1, axis=1)
            undistortedPoints2 = cv2.undistortPoints(right_cor, cameraMatrix2, distCoeffs2, P=cameraMatrix2)
    
            # Perform triangulation
            point_3d_homogeneous = cv2.triangulatePoints(P1, P2, undistortedPoints1, undistortedPoints2)
            # Convert homogeneous coordinates to Cartesian
            point_3d_cartesian = point_3d_homogeneous[:3] / point_3d_homogeneous[3]
            # Extract x, y, z coordinates
            x, y, z = point_3d_cartesian.flatten()

        #     new_row_data = {'left_id': left_id, 'right_id': row['id'],'difference': difference,'frame':row['frame']}
        #     new_row = pd.DataFrame(new_row_data,index=[0])

            df_temp = pd.DataFrame({'frame':frame,'left_id': cur_id, 'right_id': match_id,'x': x,'y': y, 'z': z},index=[0])
            df_3d_v2 = pd.concat([df_3d_v2, df_temp], ignore_index=True)
            
display(df_3d_v2)

/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_21441/440496333.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x_1 = float(df1_tt['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_21441/440496333.py:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_1 = float(df1_tt['y'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_21441/440496333.py:16: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  x_2 = float(df2_tt['x'])
/var/folders/fg/2mrxp8hx7cb9ws9lq58m21940000gn/T/ipykernel_21441/440496333.py:17: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y_2 = float(df2_tt

,frame,left_id,right_id,x,y,z
0,1,0,5.0,475.641316,-251.609266,2175.999486
1,1,1,6.0,358.941668,-255.299200,2170.809067
2,1,2,3.0,1187.891829,-270.103752,2587.299296
3,1,3,0.0,273.150068,-478.047934,2606.051411
4,1,4,2.0,634.803972,-551.119175,2809.724279
...,...,...,...,...,...,...
28126,3117,3,0.0,108.709105,-315.552335,2472.854547
28127,3117,4,2.0,116.005151,-677.619052,2495.082079
28128,3117,5,1.0,770.093280,-740.882905,2565.283206
28129,3117,6,7.0,26.176099,-131.034777,2496.313876


In [139]:
df_3d_v2.to_csv('3d_coordinatesV2.csv', index=False)

left id is chosen as new id

In [136]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from IPython.display import display

# Assuming df_3d_v2 is already defined
df = df_3d_v2

# Group the DataFrame by 'frame'
grouped = df.groupby('frame')

# Create a function to update the plot based on the selected frame
def update_plot(frame):
    data = grouped.get_group(frame)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data['x'], data['y'], data['z'], marker='o')
    
    # Annotate each point with its corresponding id
    for i, txt in enumerate(data['left_id']):
        ax.text(data['x'].iloc[i], data['y'].iloc[i], data['z'].iloc[i], str(txt), color='red')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Frame {frame}')
    plt.show()

# Create a slider widget
frame_slider = widgets.IntSlider(min=df['frame'].min(), max=df['frame'].max(), value=df['frame'].min(), description='Frame')

# Display the slider and plot
widgets.interact(update_plot, frame=frame_slider)


interactive(children=(IntSlider(value=1, description='Frame', max=3117, min=1), Output()), _dom_classes=('widg…

<function __main__.update_plot(frame)>

In [138]:
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
df = df_3d_v2
# Directory to save images
save_dir = 'animation_frames'
os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Group the DataFrame by 'frame'
grouped = df.groupby('frame')

# Define a function to save each frame as an image
def save_frame(frame):
    data = grouped.get_group(frame)
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(data['x'], data['y'], data['z'], marker='o')
    
    # Annotate each point with its corresponding id
    for i, txt in enumerate(data['left_id']):
        ax.text(data['x'].iloc[i], data['y'].iloc[i], data['z'].iloc[i], str(txt), color='red')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Frame {frame}')
    
    # Save the figure
    save_path = os.path.join(save_dir, f'frame_{frame}.png')
    plt.savefig(save_path)
    plt.close(fig)  # Close the figure to release memory

# Loop through each frame and save it
for frame in df['frame'].unique():
    save_frame(frame)


In [140]:
import cv2
import os

# Define the directory containing the frames
frames_dir = 'animation_frames'

# Define the output video file name
output_video = 'animationV2.mp4'

# Create a list to hold the paths of all frame images
frame_paths = [os.path.join(frames_dir, f) for f in os.listdir(frames_dir) if f.endswith('.png')]

# Sort the frame paths by file name
frame_paths.sort()

# Read the first frame to get dimensions
first_frame = cv2.imread(frame_paths[0])
height, width, _ = first_frame.shape

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 30, (width, height))  # Adjust FPS as needed

# Write frames to video
for frame_path in frame_paths:
    frame = cv2.imread(frame_path)
    out.write(frame)

# Release video writer
out.release()

print("Video generation complete!")


Video generation complete!
